# Imports & setup

In [6]:
# Import selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [7]:
# Import undetected chromedriver
import undetected_chromedriver as uc

In [8]:
# Import time & pandas
import time
import pandas as pd

In [10]:
# Set Chrome Options, specifically to allow location tracking on PrizePicks
capabilities = DesiredCapabilities().CHROME

chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")

prefs = {
    'profile.default_content_setting_values':
    {
        'notifications': 1,
        'geolocation': 1
    },

    'profile.managed_default_content_settings':
    {
        'geolocation': 1
    },
}

chrome_options.add_experimental_option('prefs', prefs)
capabilities.update(chrome_options.to_capabilities())

In [11]:
# Open browser
driver = uc.Chrome(options = chrome_options)

In [12]:
# Visit PrizePicks Website
driver.get("https://app.prizepicks.com/")
time.sleep(5)

In [13]:
# Close initial pop-up
WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "close")))
time.sleep(5)
driver.find_element(By.XPATH, "/html/body/div[3]/div[3]/div/div/button").click()
time.sleep(5)

In [14]:
# Navigate to COD tab
driver.find_element(By.XPATH, "//div[@class='name'][normalize-space()='COD']").click()
time.sleep(5)

In [15]:
# Initilize an empty list to hold the player props
player_props = []

In [16]:
# Get the stat container and various stats 
stat_container = WebDriverWait(driver,
                               1).until(EC.visibility_of_element_located((By.CLASS_NAME, "stat-container")))
categories = driver.find_element(By.CSS_SELECTOR, ".stat-container").text.split('\n')

In [17]:
# Remove Maps 1 - 3 Kills & Maps 1 - 3 Kills (Combo)
if "MAPS 1-3 Kills (Combo)" in categories:
    categories.remove("MAPS 1-3 Kills (Combo)")
if "MAPS 1-3 Kills" in categories:
    categories.remove("MAPS 1-3 Kills")
categories

['MAP 1 Kills', 'MAP 2 Kills', 'MAP 3 Kills']

In [18]:
# Get player projections

# Print the beginning of the logging.
print("-----------------------------")

print("Beginning Data Retrieval     ")
print("-----------------------------")
print("")

# Loop through categories
for category in categories:
    driver.find_element(By.XPATH, f"//div[text()='{category}']").click()
    time.sleep(5)

    # Test print statement
    print(f"{category}")
    
    # Get list of all projections for current category
    projectionsPP = driver.find_elements(By.ID, "test-projection-li")

    # Loop through current list of projections
    for i in range(len(projectionsPP)):

        # Get line info for each player prop
        player = projectionsPP[i].find_element(By.ID, "test-player-name").text
        team_abbr = projectionsPP[i].find_element(By.ID, "test-team-position").text.split(" - ")[0]
        player_line = float(driver.find_element(
            By.XPATH, 
            '/html/body/div[1]/div/div[3]/div[1]/div/main/div/div/div[1]/div[3]/ul/li[' + str(i + 1) + ']/div[3]/div/div/div/div[1]'
        ).text)

        # Append prop to our list
        player_props.append({
            "player": player, 
            "team_abbr": team_abbr, 
            "proptype": category.split(" ")[1], 
            "player_line": player_line
        })

        # Test print statement
        print(f"{player} {team_abbr} | {player_line}")

    # Print Formatting
    print("")
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Beginning Data Retrieval     
-----------------------------

MAP 1 Kills
Lynz MIN | 23.5
Standy MIN | 21.5
Gunless MIN | 19.5
Accuracy MIN | 20.5
Simp ATL | 25.0
aBeZy ATL | 24.0
Drazah ATL | 23.5
Cellium ATL | 22.5
Nero LV | 25.0
Purj LV | 22.0
Gio LV | 23.0
Attach LV | 23.0
Estreal LAG | 23.5
Fame LAG | 22.0
Diamondcon LAG | 22.5
Assault LAG | 20.5

MAP 2 Kills
Lynz MIN | 7.0
Standy MIN | 6.5
Gunless MIN | 5.5
Accuracy MIN | 6.0
Simp ATL | 7.5
aBeZy ATL | 7.5
Drazah ATL | 7.0
Cellium ATL | 6.5
Nero LV | 7.0
Purj LV | 6.5
Gio LV | 6.5
Attach LV | 7.0
Estreal LAG | 6.5
Fame LAG | 6.0
Diamondcon LAG | 6.5
Assault LAG | 6.0

MAP 3 Kills
Lynz MIN | 21.5
Standy MIN | 19.5
Gunless MIN | 17.5
Accuracy MIN | 18.5
Simp ATL | 23.0
aBeZy ATL | 22.0
Drazah ATL | 21.5
Cellium ATL | 20.5
Nero LV | 24.0
Purj LV | 22.0
Gio LV | 22.5
Attach LV | 23.0
Estreal LAG | 23.0
Fame LAG | 22.0
Diamondcon LAG | 22.0
Assault LAG | 20.5

-----------------------------
Data Retrieval C

In [19]:
# Convert our list of player props to a dataframe
props_df = pd.DataFrame(player_props)
props_df

,player,team_abbr,proptype,player_line
0,Lynz,MIN,MAP 1 Kills,23.5
1,Standy,MIN,MAP 1 Kills,21.5
2,Gunless,MIN,MAP 1 Kills,19.5
3,Accuracy,MIN,MAP 1 Kills,20.5
4,Simp,ATL,MAP 1 Kills,25.0
5,aBeZy,ATL,MAP 1 Kills,24.0
6,Drazah,ATL,MAP 1 Kills,23.5
7,Cellium,ATL,MAP 1 Kills,22.5
8,Nero,LV,MAP 1 Kills,25.0
9,Purj,LV,MAP 1 Kills,22.0


In [1]:
from webscraper import *

In [2]:
player_props_df = scrape_prizepicks()
player_props_df

MAP 1 Kills
MAP 2 Kills
MAP 3 Kills


,player,team_abbr,proptype,player_line
6,Clayster,CAR,1,23.5
30,Clayster,CAR,2,5.5
54,Clayster,CAR,3,21.5
7,FeLo,CAR,1,22.5
31,FeLo,CAR,2,5.5
...,...,...,...,...
43,Insight,TOR,2,6.5
67,Insight,TOR,3,20.5
18,Scrap,TOR,1,26.5
42,Scrap,TOR,2,7.0


In [3]:
from setup.setup import *